**To rerun the Notebook in Colab, save a copy in drive and change runtime type to T4 GPU**

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
squad = squad.train_test_split(test_size=0.2)

In [ ]:
squad["train"][0]

{'id': '56cd9ed662d2951400fa6783',
 'title': 'The_Legend_of_Zelda:_Twilight_Princess',
 'context': "The team worked on a Wii control scheme, adapting camera control and the fighting mechanics to the new interface. A prototype was created that used a swinging gesture to control the sword from a first-person viewpoint, but was unable to show the variety of Link's movements. When the third-person view was restored, Aonuma thought it felt strange to swing the Wii Remote with the right hand to control the sword in Link's left hand, so the entire Wii version map was mirrored.[p] Details about Wii controls began to surface in December 2005 when British publication NGC Magazine claimed that when a GameCube copy of Twilight Princess was played on the Revolution, it would give the player the option of using the Revolution controller. Miyamoto confirmed the Revolution controller-functionality in an interview with Nintendo of Europe and Time reported this soon after. However, support for the Wii c

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="BERT_BOT",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.686116


Epoch,Training Loss,Validation Loss
1,No log,1.686116
2,2.184900,1.387406
3,2.184900,1.354796


TrainOutput(global_step=750, training_loss=1.7951291097005209, metrics={'train_runtime': 948.4797, 'train_samples_per_second': 12.652, 'train_steps_per_second': 0.791, 'total_flos': 2351670810624000.0, 'train_loss': 1.7951291097005209, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/ron-the-code/my_awesome_qa_model/commit/8a04cf4422c2352911d8ddb3ba541fa0dabd3424', commit_message='End of training', commit_description='', oid='8a04cf4422c2352911d8ddb3ba541fa0dabd3424', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ron-the-code/my_awesome_qa_model', endpoint='https://huggingface.co', repo_type='model', repo_id='ron-the-code/my_awesome_qa_model'), pr_revision=None, pr_num=None)

In [ ]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

In [ ]:
from transformers import pipeline

In [ ]:
question_answerer = pipeline("question-answering", model="my_awesome_qa_model")
question_answerer(question=question, context=context)

Device set to use cuda:0


{'score': 0.28497374057769775,
 'start': 58,
 'end': 95,
 'answer': '46 languages natural languages and 13'}

In [ ]:
def qna(question, context):
  question_asnwerer=pipeline("question-answering", model="my_awesome_qa_model")
  return question_answerer(question=question, context=context)['answer']

In [ ]:
qna("What's my name?", "My name is Ron the code")

Device set to use cuda:0


'Ron'

In [ ]:
qna("How old is Taylor?", "Taylor Swift was born on December 13, 1989, in Reading, Pennsylvania. She gained fame as a country music artist before transitioning to pop music.")

Device set to use cuda:0


'December 13, 1989'

In [ ]:
qna("How old is Taylor?", "Taylor Swift was born on December 13, 1989, in Reading, Pennsylvania. She gained fame as a country music artist before transitioning to pop music. she is 18 years old")

Device set to use cuda:0


'18 years'